## Advanced Notebook

Congrats - you have made it to the advanced portion of the UCSB SQL Curriculum. By now, you should be familiar with querying up to joins and creating databases. This next section takes your SQL knowledge a level deeper, teaching you how to manipulate data and creating new columns based on your pre-existing attributes. Take a deep breath before you begin - this is going to be a long one!

First off, let's start with subqueries. 

### What are subqueries?

A subquery is a SQL statement that has another SQL query embedded in the clause. They may occur in:

- A SELECT clause
- A FROM clause
- A WHERE clause

The subquery can be nested inside a `SELECT`, `INSERT`, `UPDATE`, or `DELETE` statement or inside another subquery. A subquery is usually added within the WHERE Clause of another SQL SELECT statement.

A general subquery from the Employees table can look like:

![](img/subquery.png)

You can use the comparison operators, such as >, <, or =. The comparison operator can also be a multiple-row operator, such as IN, ANY, or ALL. In this case, we are trying to narrow down the comparison in a subquery before applying it to the overall table.

A subquery is also called an inner query or inner select, while the statement containing a subquery is also called an outer query or outer select. The inner query executes first before its parent query so that the results of an inner query can be passed to the outer query.

You can use a subquery in a `SELECT`, `INSERT`, `DELETE`, or `UPDATE` statement to perform the following tasks:
- Compare an expression to the result of the query
- Determine if an expression is included in the results of the query
- Check whether the query selects any rows

Connect to the database depending on your operating system and let's begin!

### Mac

In [ ]:
-- connection: postgresql://localhost:5432/northwind

### Windows

In [ ]:
-- connection: host='localhost' dbname='northwind' user='postgres' password='postgres'

### Let's begin!

We'll start with a simple query. 

In [ ]:
SELECT CategoryId FROM Categories
WHERE CategoryName = 'Seafood';

We will use the output of this “SELECT” query (aka the sub-query) as the input for another “SELECT” query. Let's work with the Products table, which also has a CategoryID column.

In [ ]:
SELECT * FROM Products
WHERE CategoryId =
(SELECT CategoryId FROM Categories
WHERE CategoryName = 'Seafood');

Does the logic behind this query look familiar? 

We can also write these queries as table joins! Let's write the queries with a slightly different syntax.

In [ ]:
SELECT Products.*
FROM Products, Categories
WHERE Products.CategoryId = Categories.CategoryId
AND CategoryName = 'Seafood';

This implicit inner join outputs the same query as the subquery. We are pulling the same criteria from both tables and the only difference is the syntax! This is not the case for all subqueries, but an example of when subqueries can be useful.

### Subqueries for Comparison

Sub-queries can also be used with other comparison operators >, <, >=, <=, etc.
Starting with the inner query:

In [ ]:
SELECT AVG(UnitPrice) FROM Products;

This subquery returns a singular (scalar) value. So when we create the entire query, we are comparing against this one value.

In [ ]:
SELECT * FROM Products
WHERE UnitPrice >
(SELECT AVG(UnitPrice) FROM Products);

This is a case where there is no equivalent table join format. 

### Multiple Subqueries 

We can continue to create deeper subqueries to draw in the information we need. Let's build this step-by-step.

First, we'll create a simple query. 

In [ ]:
SELECT * FROM Employees 
WHERE title LIKE '%Sales%'; 

Say we want to build off this table and work with it. Then the syntax goes:

```
SELECT * FROM
(SELECT * FROM inner_query) [AS] foo
[Constraints]
```

We alias the table so that as we build on it, we can add further constraints on your query (i.e., `WHERE` clauses). Otherwise the database will not know how to refer to the subquery.

Using the same simple query we created above, let's use a subquery to count how many sales representatives there are. 

In [ ]:
SELECT COUNT(title)
FROM
(SELECT * FROM Employees 
WHERE title LIKE '%Sales%') AS Employees 
WHERE title LIKE '%Rep%';

Here's the catch - we know we can do this in one step. Can you write that query down below? 

This is a quick example of what a nested subquery, but not one that was necessarily what we needed to answer our question. 
You will soon see why we need multiple layers of queries soon. But first, let's practice!

### Extra Practice

What are the units in stock from the Products table where the unit price is not equal to the max unit price from the Order Details table? 

**(Hint: Put the subquery in the `Where` clause.)**

Find all the columns in the Orders table where the CustomerID is equal to the contact name `Hanna Moos`. 

**(Hint: Use the Customers table to find the contact name and put it in the subquery.)**

Answer the same question as above using a join.

### Window Functions

Window functions are extremely powerful SQL commands that perform a calculation across a set of table rows that are somehow related to the current row. This is comparable to the type of calculation that can be done with an aggregate function. 

But unlike regular aggregate functions, use of a window function does not cause rows to become grouped into a single output row — the rows retain their separate identities. Behind the scenes, the window function is able to access more than just the current row of the query result.

Window functions can be called in the SELECT statement or in the ORDER BY clause. However, they can never be called in the WHERE clause. You’ll notice that all the examples in this article call the window function in the SELECT column list.


The basic syntax goes like:

```
SELECT	
		[attributes],
        WINDOW_FUNCTION() OVER (PARTITION BY attribute ORDER BY attribute DESC) 
			AS alias
FROM table_name;
```


Let's use an exerpt from the Employees table as our example.

|employee_id|full_name| department | salary| 
|---|--- |---| --- |
|100|Mary Johns| SALES |1000.00| 
|101|Sean Moldy| IT |1500.00| 
|102|Peter Dugan| SALES |2000.00|
|103|Lilian Penn| SALES |1700.00|
|104|Milton Kowarsky| IT |1800.00|
|105|Mareen Bisset| ACCOUNTS |1200.00|
|106|Airton Graue| ACCOUNTS |1100.00|

Window functions start with RANK, which is one of the simplest window functions. It returns the position of any row inside the partition. Let’s use it to rank salaries within departments:

```
SELECT	
		employee_id, 
		full_name, 
		salary,
        department,
        RANK() OVER (PARTITION BY department ORDER BY salary DESC) 
			AS dept_ranking	
FROM Employee;
```


|employee_id|full_name| salary|department| dept_ranking |
|---|--- |---| --- | ---|
|105|Mareen Bisset| 1200.00| ACCOUNTS |1 |
|106|Airton Graue| 1100.00| ACCOUNTS |2|
|104|Milton Kowarsky| 1800.00| IT |1|
|101|Sean Moldy| 1500.00| IT | 2|
|102|Peter Dugan| 2000.00| SALES |1|
|103|Lilian Penn| 1700.00 |SALES |2|
|100|Mary Johns| 1000.00| SALES |3 | 

We have created a `dept_ranking` column that first groups the departments, and then ranks them based on salary. 

Now, let's run through some actual code examples

Suppose we want to know the price of a product with the discount. This calls for a math expression, like:

`unitprice - discount`

We will want to subtract the cumulative amount of discounts from the unitprice. 
The next query will show all order details and the `sale_price` of the product. Note that we don't want where the product has no discounts - that's lame!

In [ ]:
SELECT *, 
        unitprice - sum(discount) OVER(PARTITION BY orderid ORDER BY productid) AS sale_price
FROM Order_Details
WHERE discount != 0;

This new column `sale_price` is a result of our window function. This is where the power of SQL comes in - we can manipulate our data to output new tables and aggregate columns as we see fit.

There are many different window functions to run through. Here are the most common window functions, separated by their types:

#### Value

- FIRST_VALUE()
- LAG()
- LAST_VALUE()
- LEAD()

#### Aggregate

- AVG()
- COUNT()
- MAX()
- MIN()
- SUM()

#### Ranking

- CUME_DIST()
- DENSE_RANK()
- NTILE()
- PERCENT_RANK()
- RANK()
- ROW_NUMBER()

Let's run through another example together.

Starting with the Employees table, let's see the Employee ID, last name, first name, birth date, and hire date. Go ahead and create that table first.

Next we'll be creating a new column that outputs the rank of each employee based on their hire date, so we can see how long each employee started working. Check out the output below to see how it is done. 

In [ ]:
SELECT employeeid, lastname, firstname, birthdate, hiredate, 
    RANK() OVER(PARTITION BY hiredate ORDER BY employeeid) AS hire_number
FROM Employees;

Each hire was the only one during its date, that's why the column outputs `1`,  *EXCEPT* Michael Suyama and Steven Buchanan, who were hired on the same day. Steven was hired first, as Employee ID `5`, so he gets to be hire_number 1.

### Extra Practice

Select the orderid, customerid, shipvia, and freight from the Orders table. Create a column takes the max_freight partitioned by the customerid, and ordered by the shipvia attribute. Call this column `max_freight`. 

Output the company name, contact name, contact title from the Suppliers table and the product ID and the product name from the Products table where the contact title is in Marketing. Order the table alphabetically by company name.

### Window Functions in Subqueries

In this section, we will introduce why multiple layers of queries are needed. You can build on your subqueries as you create more columns and your table begins to contain more nuanced attributes that you will continue to reference in your querying. Let's explain this with an example.

Start with the Order Details table.

In [ ]:
SELECT * FROM Order_Details;

Let's say we want to create a column with the min unit price depending on the product ID. We can drop the Order ID, quantity, and discount columns. That query would look something like this:

In [ ]:
SELECT productid, unitprice,
    MIN(unitprice)OVER(PARTITION BY productid ORDER BY orderid) as min
FROM Order_Details;

There are clearly a ton of duplicates in this new table, and we don't need each row. So we need to add some conditions. But here's the catch - we can't add any clauses on the `min` column in this initial query, because it doesn't exist yet. We are using this particular statement to *create* the column. 

We will work together to pick the `min` value that corresponds with the minimum unit price and only output that single row, breaking the process down step by step. If we want to rank the min values for each product (id), it would look like the query below. Don't forget the alias!

In [ ]:
SELECT productid, unitprice, min, 
 DENSE_RANK()OVER(PARTITION BY productid ORDER BY unitprice) as rank  
FROM
(SELECT *,
    MIN(unitprice)OVER(PARTITION BY productid ORDER BY orderid) as min
FROM Order_Details) as first_subquery
ORDER BY productid;

One way to test if this worked is to only look at one product id. Modify our query in the box down below so that we are only looking at Product Id `45`.

How does it look? Does everything make sense? 

Since we are only looking at the min values, we really only want the row with the lowest unitprice. This corresponds with row_num `1`. Therefore, we need to add another layer to our subquery, choosing ONLY the rows that have row_num `1`. Remember, we can't do that directly to our current query since the `rank` column is still being created! The query now looks like this:

In [ ]:
SELECT DISTINCT productid, unitprice, min, rank 
FROM
(SELECT *,
    DENSE_RANK()OVER(PARTITION BY productid ORDER BY unitprice) as rank  
FROM
(SELECT *,
    MIN(unitprice)OVER(PARTITION BY productid ORDER BY orderid) as min
FROM Order_Details) as first_subquery
ORDER BY productid) as second_subquery
WHERE rank = '1'
ORDER BY productid;

Scan through the newly created table quickly. Does each unitprice correspond with each min? 

This is an example of how deep subqueries can get, with adding and modifying columns. The layers can get tricky! Hopefully walking through an example together helps - we start with the simplest query we can get and build off it, creating clauses and subqueries where necessary. This is the secret to writing in SQL, break down each step and work your way to the finished product!

### THE END!

You have officially reached the end of the UCSB Data Science Club SQL tutorial. Congratulations on making it this far - everything you have learned in these notebooks will prepare you for advanced querying. Now that you’ve got a handle on SQL, you have the skills to navigate an entire database, create a database, and modify tables to create new data! These are some pretty solid skills you now have under your belt. 

Thank you to everyone who helped make this workshop and this tutorial possible. The material draws heavily from [w3schools' SQL tutorial](https://www.w3schools.com/sql/) and is a great resource to check out to learn more. The three notebooks were written by [Samantha Lee](http://github.com/samanthaklee/) and the seamless installation process was created by [Timothy Nguyen](https://github.com/timothydnguyen/), [Raul Eulogio](https://github.com/raviolli77), and [Shon Inouye](https://github.com/inouyesan). Finally, shout out to the four PSTAT clubs who banded together in the epic crossover event of the year. Check out each of our pages for more details!    

[Data Science @ UCSB](https://www.facebook.com/DataScienceUCSB/)

[Actuarial Association](http://actuaryclub.pstat.ucsb.edu/)

[FACTOR](https://www.facebook.com/FACTORatUCSB/)

[UCSB FAME](https://www.facebook.com/FAMEUCSB/)

Here are some extra practice problems to end the final notebook. Good luck with the rest of your SQL journey!

### Extra Practice

**Part A.** Create a column that outputs the average units on order from the Products table. Partition by the category ID, and don't include any units on order that are 0's. 

**Part B.** Rank the last query over the category id and order by the unit price. Alias the rank as `rank` and call the first subquery `first_subquery.`

**Part C.** 
Ouput all columns from the previous query where the rank is `2`.

**Part A.** Select the customer ID, company name, contact name from the Customers table and select the employee ID and freight from the Orders table. Join these two tables with a left join and order by the employee ID.

**Part B.** Use the `SUM` window function to find the total freight for each employee ID. Alias this as sum_freight.

**(Hint: You can add this as a line to the initial query.)**

**Part C.** Filter the table to output only the sum freight that corresponds with the minimum freight. Include employee ID null values. 

**(Hint: This can be done with one subquery)**

What does the query look like for this table? 

|lastname|firstname| shipvia|shipvia_row| 
|---|--- |---| --- |
|Buchanan|Steven| 1| 1 |
|Buchanan|Steven| 2| 1 |
|Buchanan|Steven| 3| 1 |
|Fuller|Andrew| 1| 90 | 
|Fuller|Andrew| 2| 127 |
|Fuller|Andrew| 3 |98 |

**(Hint: Inner join the Employees table and the Orders table.)**